In [1]:
from langchain.llms import OpenAI

In [52]:
import os
os.environ['OPENAI_API_KEY']='1234'

In [42]:
llm=OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],temperature=0.7)

In [6]:
text = "what is the capital of india"

print(llm(text))



The capital of India is New Delhi.


In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]='hf_rWtKlWTEUGokXqlQLdlXUrxbv'

In [10]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id='google/flan-t5-base',model_kwargs={"temperature":0, "max_length": 64})

In [17]:
output = llm_huggingface.predict("what is the capital of India")
print(output)

chennai


### Prompt Template and LLMChain

In [21]:
from langchain.prompts import PromptTemplate
template = "what is the capital of {country}"
prompt_template = PromptTemplate(input_variables=['country'],template=template)
prompt_template.format(country="india")

'what is the capital of india'

In [29]:
# from langchain.chains import LLMChain
# chain=LLMChain(llm=llm,prompt=prompt_template)
# print(chain.run("India"))

# A good way to write the same code

chain = prompt_template | llm
res = chain.invoke({"country":"india"})
print(res)



The capital of India is New Delhi. 


### Combining Multiple Chains Uing simple Sequential Chain

In [31]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)

capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

In [32]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [33]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

" Here are some amazing places to visit in and around New Delhi:\n\n1. Red Fort - a historic fort made of red sandstone, built in the 17th century by the Mughal emperor Shah Jahan.\n\n2. India Gate - a war memorial dedicated to the soldiers who died in World War I and the Afghan Wars.\n\n3. Qutub Minar - a UNESCO World Heritage Site and the tallest brick minaret in the world, built in the 12th century.\n\n4. Humayun's Tomb - a magnificent tomb built for the Mughal emperor Humayun, with beautiful gardens and architecture.\n\n5. Lotus Temple - a Bahá'í House of Worship known for its unique lotus-shaped design and peaceful atmosphere.\n\n6. Akshardham Temple - a Hindu temple complex known for its stunning architecture and spiritual ambiance.\n\n7. Chandni Chowk - one of the oldest and busiest markets in Old Delhi, known for its vibrant atmosphere and delicious street food.\n\n8. Connaught Place - a popular shopping and dining hub in New Delhi, with a mix of modern and colonial architectur

### Sequential Chain

In [35]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [36]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [37]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [38]:
chain({'country':"India"})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Here are some amazing places to visit in New Delhi:\n\n1. Red Fort: This iconic monument is a must-visit for its beautiful Mughal architecture and historical significance.\n\n2. India Gate: This war memorial is a popular spot for picnics and evening strolls, and is lit up beautifully at night.\n\n3. Qutub Minar: This UNESCO World Heritage Site is the tallest brick minaret in the world and a stunning example of Indo-Islamic architecture.\n\n4. Lotus Temple: This architectural marvel is a Bahá'í House of Worship and a peaceful oasis in the bustling city.\n\n5. Humayun's Tomb: Another UNESCO World Heritage Site, this tomb is the final resting place of Mughal Emperor Humayun and a beautiful example of Mughal architecture.\n\n6. Jama Masjid: This grand mosque is one of the largest in India and a popular spot for photography and cultural immersion.\n\n7. Chandni Chowk: This bustling market is a must-visit

In [39]:
### Chatmodels With ChatOpenAI

In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [54]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [55]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "Why did the AI cross the road? To optimize its path-finding algorithm!"\n2. "I asked my AI assistant to tell me a joke, and it replied, \'I\'m sorry, I\'m not programmed for humor... but I can recommend a good comedy show!\'"\n3. "I told my AI to make me laugh, and it responded with a binary joke: \'There are 10 types of people in the world: those who understand binary and those who don\'t.\'"\n4. "My AI assistant keeps trying to cheer me up with jokes, but its sense of humor is a bit artificial."\n5. "My AI\'s idea of a knock-knock joke: \'Knock, knock.\' \'Who\'s there?\' \'Error 404: Joke not found.\'"\n6. "I asked my AI to tell me a joke about machine learning, and it replied, \'I\'m still training on that one...\'"\n7. "Why did the AI go to therapy? It had too many unresolved issues and couldn\'t debug itself!"')

### Prompt Template + LLM + OutputParser

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [64]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [65]:
template="Your are a helpful assistant. When the user given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [66]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [67]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' bright', ' astute', ' sharp']